<a href="https://colab.research.google.com/github/YunusKhan/NectarineDemo/blob/master/Nectarine_102_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.regularizers import  l1_l2



In [53]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
lframe = []
path = "/content/drive/My Drive/FORTH_TRACE_DATASET-master/*/*.csv"
for f in glob.glob(path):
    lframe.append(pd.read_csv(f, header=None))
df = pd.concat(lframe, ignore_index=False, sort=False, axis=0)



In [0]:
df = df[ df[0] == 2]  # dropping all other positions based data other than right wrist 
df = df[ df[11] <= 5] # dropping all other activites other than 1-5 

del df[0] # removing the first column because it is only right wrist and is dead weight 

X_scaled = preprocessing.scale(df[10].values)  # normalising the data on  time stamp as it is very varied and fluctuating so keeping it between -1 & 1
# may move it to 0 to 1 or some other thing later, right or some other scaling 
df[10] = X_scaled

In [0]:
x = df.iloc[:,:-1]
y = df.iloc[:,10:].values.ravel()
#x = x.values.reshape((x.shape[0], x.shape[1], 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [57]:
model = Sequential()
model.add(Dense(1024, input_dim=10, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, batch_size = 1024, epochs = 25, verbose = 2, validation_data = (x_test, y_test))
dense_dl_score = model.evaluate(x_test, y_test)
print ( 'Dense DL  ... ', dense_dl_score)
#print ( 'Dense DL Accuracy ... ', dense_dl_score[1])

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 1024)              11264     
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 1025      
Total params: 12,289
Trainable params: 12,289
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
473/473 - 4s - loss: 2.1762 - accuracy: 0.2568 - val_loss: 0.5533 - val_accuracy: 0.2610
Epoch 2/25
473/473 - 4s - loss: 1.2820 - accuracy: 0.2595 - val_loss: 0.5633 - val_accuracy: 0.2610
Epoch 3/25
473/473 - 4s - loss: 0.9741 - accuracy: 0.2599 - val_loss: 0.6896 - val_accuracy: 0.2610
Epoch 4/25
473/473 - 4s - loss: 0.8103 - accuracy: 0.2607 - val_loss: 0.9370 - val_accuracy: 0.2610
Epoch 5/25
473/473 - 4s - loss: 0.7090 - accuracy: 0.2608 - val_loss: 0.6509 - val_accuracy: 0.2610
Epoch 

In [58]:
print ( "looks like a normal minimum dense netork does not work, flat accuracy. was just trying out my luck.. let me see other options as well")

looks like a normal minimum dense netork does not work, flat accuracy. was just trying out my luck.. let me see other options as well


In [0]:
x = df.iloc[:,:-1]
y = df.iloc[:,10:].values.ravel()
x = x.values.reshape((x.shape[0], x.shape[1], 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [62]:
model = Sequential()
model.add(Conv1D ( filters = 128, kernel_size = 3, activation = 'relu', input_shape = (10,1)))
model.add(Conv1D ( filters = 128, kernel_size = 3, activation = 'relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

print(model.summary())
model.fit(x_train, y_train, batch_size = 1024, epochs = 25, verbose = 2, validation_data = (x_test, y_test))
dense_dl_score = model.evaluate(x_test, y_test)
print ( 'Dense DL  ... ', dense_dl_score[0])
print ( 'Dense DL Accuracy ... ', dense_dl_score[1])

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_14 (Conv1D)           (None, 8, 128)            512       
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 6, 128)            49280     
_________________________________________________________________
dropout_7 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 3, 128)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 384)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               49280     
Total params: 99,072
Trainable params: 99,072
Non-trainable params: 0
_________________________________________________

In [0]:
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3,padding='valid', activation='relu',input_shape=(10,1), strides = 1,  kernel_regularizer=l1_l2(l1=0.05, l2=0.05)))
model.add(MaxPooling1D(2,data_format='channels_last'))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3 , padding='valid', activation='relu',kernel_regularizer = l1_l2(l1=0.05, l2=0.05), strides=1))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation = 'softmax'))
model.compile(loss='squared_hinge', optimizer='adadelta', metrics=['accuracy'])
print(model.summary())
model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2, verbose=2, shuffle=False )
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



ERROR! Session/line number was not unique in database. History logging moved to new session 60
Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_36 (Conv1D)           (None, 8, 128)            512       
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 4, 128)            0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 4, 128)            0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 4, 128)            512       
_________________________________________________________________
conv1d_37 (Conv1D)           (None, 2, 128)            49280     
_________________________________________________________________
dropout_39 (Dropout)         (None, 2, 128)            0         
________________________